In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tsgm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
#Save Kyoto_Gases data (2020-2100), consider the case of C1-C8 for simplicity.
Kyoto_Gases = pd.read_csv('Kyoto Gases.csv')
Kyoto_Gases = Kyoto_Gases[Kyoto_Gases['Category'].isin(['C1','C2','C3','C4','C5','C6','C7','C8'])]
mapping = {'C1':0,'C2':0,'C3':0,'C4':1,'C5':1,'C6':1,'C7':2,'C8':2}
Kyoto_Gases['Category'].replace(mapping,inplace=True)
Kyoto_Gases.reset_index(drop=True,inplace=True)
Kyoto_Gases.drop(columns=['Category_name'],inplace = True)

In [3]:
#Load a dataset of individual variables
CarbonSequestration = pd.read_csv('Carbon_Sequestration_CCS_imputed.csv')
FinalEnergy_Liquid = pd.read_csv('Final Energy_Liquids.csv')
PrimaryEnergy_Gas = pd.read_csv('Primary Energy_Gas.csv')
PrimaryEnergy_Oil = pd.read_csv('Primary Energy_Oil.csv')
PrimaryEnergy_Coal = pd.read_csv('PrimaryEnergy_Coal.csv')

In [4]:
#Get the intersection of the models and scenarios contained in each variable
Model_Scenario = Kyoto_Gases[['Model','Scenario']]
Variables = [CarbonSequestration,FinalEnergy_Liquid,PrimaryEnergy_Coal,PrimaryEnergy_Gas,PrimaryEnergy_Oil]
for variable in Variables:
    Model_Scenario = pd.merge(Model_Scenario,variable[['Model','Scenario']],on=['Model','Scenario'],how='inner')

In [5]:
for i in range(len(Variables)):
    Variables[i] = pd.merge(Model_Scenario,Variables[i],on=['Model','Scenario'],how='inner')
for i in range(len(Variables)):
    Variables[i].drop(columns=['Category_name'],inplace = True)

In [6]:
Kyoto_Gases = pd.merge(Kyoto_Gases,Model_Scenario,on = ['Model','Scenario'],how = 'inner')

In [7]:
Variables.append(Kyoto_Gases)
#Variables [CarbonSequestration,FinalEnergy_Liquid,PrimaryEnergy_Coal,PrimaryEnergy_Gas,PrimaryEnergy_Oil,Kyoto_Gases]

In [8]:
#Generate feature matrices, the values of each variable during 2020-2100. 9 time steps, 6 features
#1160 is the amount of data
X = np.zeros((1160,9,6))
for i in range(len(Variables)):
    Variables[i] = Variables[i].iloc[:,3:-1].values
for i in range(1160):
    for j in range(9):
        for k in range(6):
            X[i][j][k] = (Variables[k])[i,j]

In [9]:
Y = Kyoto_Gases['Category'].values

In [10]:
#Separate datasets by category. But the training process does not distinguish between categories like VAE
C123_DataSet = X[Y == 0]
C456_DataSet = X[Y == 1]
C78_DataSet = X[Y == 2]

In [11]:
#Set problem parameters
latent_dim = 36
output_dim = 3
feature_dim = 6
seq_len = 9
batch_size = 100
generator_in_channels = latent_dim + output_dim
discriminator_in_channels = feature_dim + output_dim

In [12]:
#Standardize the data and compress it to (0, 1).
scaler = tsgm.utils.TSFeatureWiseScaler((0,1))
X_train = scaler.fit_transform(X)
Y_train = keras.utils.to_categorical(Y, 3)

X_train = X_train.astype(np.float32)
Y_train = Y_train.astype(np.float32)

dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

In [13]:
architecture = tsgm.models.architectures.cGAN_LSTMnArchitecture(
    seq_len=seq_len, feat_dim=feature_dim,
    latent_dim=latent_dim, output_dim=output_dim)
discriminator, generator = architecture._discriminator, architecture._generator

In [14]:
#Define an optimization strategy
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,#1e-3
    decay_steps=200,#200
    decay_rate=0.95)#0.95
cond_gan = tsgm.models.cgan.ConditionalGAN(
    discriminator=discriminator, generator=generator, latent_dim=latent_dim
)
cond_gan.compile(
    d_optimizer=keras.optimizers.Adam(lr_schedule),
    g_optimizer=keras.optimizers.Adam(lr_schedule),
    loss_fn=keras.losses.BinaryCrossentropy(),
)
cond_gan.fit(dataset, epochs=1200)

Epoch 1/1200
12/12 [==============================] - 4s 24ms/step - g_loss: 0.6870 - d_loss: 0.6787
Epoch 2/1200
12/12 [==============================] - 0s 23ms/step - g_loss: 0.5676 - d_loss: 0.7558
Epoch 3/1200
12/12 [==============================] - 0s 24ms/step - g_loss: 0.9137 - d_loss: 0.6672
Epoch 4/1200
12/12 [==============================] - 0s 24ms/step - g_loss: 1.1758 - d_loss: 0.5870
Epoch 5/1200
12/12 [==============================] - 0s 25ms/step - g_loss: 1.1451 - d_loss: 0.5435
Epoch 6/1200
12/12 [==============================] - 0s 25ms/step - g_loss: 1.3137 - d_loss: 0.4191
Epoch 7/1200
12/12 [==============================] - 0s 25ms/step - g_loss: 1.2440 - d_loss: 0.3775
Epoch 8/1200
12/12 [==============================] - 0s 27ms/step - g_loss: 0.7326 - d_loss: 0.4375
Epoch 9/1200
12/12 [==============================] - 0s 25ms/step - g_loss: 0.8896 - d_loss: 0.3219
Epoch 10/1200
12/12 [==============================] - 0s 25ms/step - g_loss: 0.1657 - d_lo

12/12 [==============================] - 0s 25ms/step - g_loss: 0.7206 - d_loss: 0.6917
Epoch 161/1200
12/12 [==============================] - 0s 25ms/step - g_loss: 0.7086 - d_loss: 0.6962
Epoch 162/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6918 - d_loss: 0.6957
Epoch 163/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6827 - d_loss: 0.6914
Epoch 164/1200
12/12 [==============================] - 0s 25ms/step - g_loss: 0.6765 - d_loss: 0.6917
Epoch 165/1200
12/12 [==============================] - 0s 27ms/step - g_loss: 0.6729 - d_loss: 0.6969
Epoch 166/1200
12/12 [==============================] - 0s 28ms/step - g_loss: 0.6816 - d_loss: 0.6997
Epoch 167/1200
12/12 [==============================] - 0s 28ms/step - g_loss: 0.7068 - d_loss: 0.6947
Epoch 168/1200
12/12 [==============================] - 0s 28ms/step - g_loss: 0.7267 - d_loss: 0.6903
Epoch 169/1200
12/12 [==============================] - 0s 28ms/step - g_loss: 0.7237 - 

12/12 [==============================] - 0s 27ms/step - g_loss: 0.6904 - d_loss: 0.6934
Epoch 319/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6883 - d_loss: 0.6930
Epoch 320/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6875 - d_loss: 0.6930
Epoch 321/1200
12/12 [==============================] - 0s 28ms/step - g_loss: 0.6883 - d_loss: 0.6934
Epoch 322/1200
12/12 [==============================] - 0s 27ms/step - g_loss: 0.6906 - d_loss: 0.6939
Epoch 323/1200
12/12 [==============================] - 0s 28ms/step - g_loss: 0.6939 - d_loss: 0.6942
Epoch 324/1200
12/12 [==============================] - 0s 27ms/step - g_loss: 0.6972 - d_loss: 0.6945
Epoch 325/1200
12/12 [==============================] - 0s 27ms/step - g_loss: 0.6992 - d_loss: 0.6945
Epoch 326/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6997 - d_loss: 0.6941
Epoch 327/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6991 - 

12/12 [==============================] - 0s 27ms/step - g_loss: 0.6964 - d_loss: 0.6939
Epoch 477/1200
12/12 [==============================] - 0s 28ms/step - g_loss: 0.6968 - d_loss: 0.6938
Epoch 478/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6964 - d_loss: 0.6937
Epoch 479/1200
12/12 [==============================] - 0s 27ms/step - g_loss: 0.6955 - d_loss: 0.6936
Epoch 480/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6942 - d_loss: 0.6936
Epoch 481/1200
12/12 [==============================] - 0s 27ms/step - g_loss: 0.6925 - d_loss: 0.6935
Epoch 482/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6910 - d_loss: 0.6933
Epoch 483/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6900 - d_loss: 0.6932
Epoch 484/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6895 - d_loss: 0.6931
Epoch 485/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6895 - 

12/12 [==============================] - 0s 27ms/step - g_loss: 0.6911 - d_loss: 0.6935
Epoch 635/1200
12/12 [==============================] - 0s 27ms/step - g_loss: 0.6921 - d_loss: 0.6935
Epoch 636/1200
12/12 [==============================] - 0s 27ms/step - g_loss: 0.6933 - d_loss: 0.6933
Epoch 637/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6946 - d_loss: 0.6932
Epoch 638/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6958 - d_loss: 0.6932
Epoch 639/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6967 - d_loss: 0.6932
Epoch 640/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6972 - d_loss: 0.6934
Epoch 641/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6972 - d_loss: 0.6936
Epoch 642/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6968 - d_loss: 0.6937
Epoch 643/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6961 - 

12/12 [==============================] - 0s 27ms/step - g_loss: 0.6853 - d_loss: 0.6934
Epoch 793/1200
12/12 [==============================] - 0s 27ms/step - g_loss: 0.6854 - d_loss: 0.6933
Epoch 794/1200
12/12 [==============================] - 0s 27ms/step - g_loss: 0.6860 - d_loss: 0.6933
Epoch 795/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6871 - d_loss: 0.6932
Epoch 796/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6887 - d_loss: 0.6932
Epoch 797/1200
12/12 [==============================] - 0s 27ms/step - g_loss: 0.6907 - d_loss: 0.6933
Epoch 798/1200
12/12 [==============================] - 0s 29ms/step - g_loss: 0.6931 - d_loss: 0.6934
Epoch 799/1200
12/12 [==============================] - 0s 28ms/step - g_loss: 0.6952 - d_loss: 0.6935
Epoch 800/1200
12/12 [==============================] - 0s 31ms/step - g_loss: 0.6972 - d_loss: 0.6935
Epoch 801/1200
12/12 [==============================] - 0s 30ms/step - g_loss: 0.6988 - 

12/12 [==============================] - 0s 29ms/step - g_loss: 0.6867 - d_loss: 0.6932
Epoch 951/1200
12/12 [==============================] - 0s 28ms/step - g_loss: 0.6871 - d_loss: 0.6932
Epoch 952/1200
12/12 [==============================] - 0s 27ms/step - g_loss: 0.6877 - d_loss: 0.6933
Epoch 953/1200
12/12 [==============================] - 0s 27ms/step - g_loss: 0.6884 - d_loss: 0.6934
Epoch 954/1200
12/12 [==============================] - 0s 27ms/step - g_loss: 0.6893 - d_loss: 0.6934
Epoch 955/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6901 - d_loss: 0.6936
Epoch 956/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6912 - d_loss: 0.6937
Epoch 957/1200
12/12 [==============================] - 0s 27ms/step - g_loss: 0.6924 - d_loss: 0.6938
Epoch 958/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6935 - d_loss: 0.6938
Epoch 959/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6950 - 

12/12 [==============================] - 0s 26ms/step - g_loss: 0.6927 - d_loss: 0.6933
Epoch 1108/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6927 - d_loss: 0.6933
Epoch 1109/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6928 - d_loss: 0.6933
Epoch 1110/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6929 - d_loss: 0.6933
Epoch 1111/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6929 - d_loss: 0.6933
Epoch 1112/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6929 - d_loss: 0.6933
Epoch 1113/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6930 - d_loss: 0.6933
Epoch 1114/1200
12/12 [==============================] - 0s 27ms/step - g_loss: 0.6930 - d_loss: 0.6933
Epoch 1115/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 0.6931 - d_loss: 0.6933
Epoch 1116/1200
12/12 [==============================] - 0s 26ms/step - g_loss: 

In [15]:
#Set Global Random Seed
np.random.seed(8)
Gen_Labels = np.zeros(1500)
Gen_Labels[500:1000] = 1
Gen_Labels[1000:] = 2
Gen_Labels.astype(np.int32)

array([0, 0, 0, ..., 2, 2, 2])

In [16]:
#Generate data using generative models (500 for each class)
z = np.concatenate([np.random.randn(1500,36),keras.utils.to_categorical(Gen_Labels,3)],axis=1)
Gen_Data = generator(z)
Gen_Data = Gen_Data.numpy()
Gen_Data = scaler.inverse_transform(Gen_Data)

In [17]:
#generator.save_weights('Rcgan_generator.h5')
#discriminator.save_weights('Rcgan_discriminator.h5')